In [1]:
from itertools import chain
from pathlib import Path

import matlab.engine

from skrt import Patient
from skrt.registration import get_default_pfiles_dir, Registration, set_elastix_dir

# Define paths to patient data.
# Each dataset consists of the CT image used in treatment planning,
# and the associated structure set.
# In each structure set, IMNs are outlined on the patient's left or right only.
# The side on which the IMNs are outlined is indicated in the name of the
# directory containing the patient directories.
data_dir = Path("~/data/casscade").expanduser()
paths = sorted(list(chain(data_dir.glob('casscade*/[!.]*'))))

# Set path to Elastix installation.
elastix_dir = Path('~/sw/elastix-5.0.1').expanduser()
set_elastix_dir(elastix_dir)

# Obtain path to default Elastix parameter files.
pfiles_dir = get_default_pfiles_dir()

# Indicate whether to print information for checking progress and data content.
verbose = True

In [2]:
# Load data.
patients = {"left": [], "right": []}
ids = []
for path in paths:
    if ids and path.name not in ids:
        continue
    # Determine side on which IMNs are outlined from path.
    side = path.parent.name.split("_")[1]
    patients[side].append(Patient(path, unsorted_dicom=True))
    if verbose:
        print(f"({side}) {patients[side][-1].id} - initialisation; "
              f"{patients[side][-1]._init_time:.2f} s; ")

(left) z0k0jwfmkl - initialisation; 0.55 s; 
(left) z0k0kwmghm - initialisation; 0.49 s; 
(left) z0k0lufhhn - initialisation; 0.43 s; 
(left) z0k0lzkfkp - initialisation; 0.58 s; 
(left) z0k0lzmggq - initialisation; 0.56 s; 
(left) z0k0mzjjij - initialisation; 0.57 s; 
(left) z0k0nrmkko - initialisation; 0.45 s; 
(left) z0k0nshmjp - initialisation; 0.48 s; 
(left) z0k0nsmjnl - initialisation; 0.53 s; 
(left) z0k0nvmgkq - initialisation; 0.52 s; 
(left) z0k0pxfejq - initialisation; 0.64 s; 
(left) z0k0q1ogkn - initialisation; 0.50 s; 
(left) z0k0qzkekh - initialisation; 0.44 s; 
(left) z0k1kwihfp - initialisation; 0.49 s; 
(left) z0k1luinem - initialisation; 0.57 s; 
(left) z0k1nxmgeo - initialisation; 0.40 s; 
(left) z0k1ozngil - initialisation; 0.51 s; 
(left) z0k2irkmnn - initialisation; 0.57 s; 
(left) z0k2rtkjkh - initialisation; 0.46 s; 
(left) z0k3jtihlp - initialisation; 0.72 s; 
(left) z0k3lxmiij - initialisation; 0.57 s; 
(left) z0k3nsmelj - initialisation; 0.70 s; 
(left) z0k

In [3]:
# For each patient, check that number of images and number of structure sets
# are both equal to 1, then print names of all ROIs
# and names or OARs to be considered in auto-segmentation.
all_oar_names = set()
for side in patients:
    if verbose:
        print(f"\n{side}: {len(patients[side])}")
    for patient in patients[side]:
        images = patient.combined_objs("image_types")
        assert 1 == len(images)
        structure_sets = patient.combined_objs("structure_set_types")
        assert 1 == len(structure_sets)
        
        ss = structure_sets[0]
        roi_names = [roi_name for roi_name in sorted(ss.get_roi_names())]
        oar_names = [roi_name for roi_name in roi_names if True in
                     [oar_name in roi_name for oar_name in
                      ["hear", "Heart", "inm", "imn", "INM", "IMN"]]]
        if verbose:
            print(f"\n({side}) {patient.id} - study timestamp: {patient.studies[0].timestamp}")
            print(f"    {roi_names}")
            print(f"    {oar_names}")
        all_oar_names = all_oar_names.union(oar_names)   
        
all_oar_names = sorted(list(all_oar_names))
if verbose:
    print(f"\n{all_oar_names}")
        
"""
idx = 0
patient = patients[idx]
study = patient.studies[0]
ct = study.ct_images[0]
ss = study.ct_structure_sets[0]
ct.view(rois=ss)
""";
assert True == False


left: 35

(left) z0k0jwfmkl - study timestamp: 20200901_145209
    ['CTV INM', 'CTV intercept', 'CTV level 2', 'CTV level 3', 'CTV level 4', 'Heart', 'L_Lung', 'Oesophagus', 'PTVn_LN_Ax_L2_L', 'PTVn_LN_Ax_L3_L', 'PTVn_LN_Ax_L4_L', 'PTVn_LN_Combined', 'PTVn_LN_IMN_L', 'PTVn_LN_Intercept_L', 'PTVp_4000', 'R_Breast', 'R_Lung', 'Skin', 'Skin-PTVAndAx', 'ptv imn dvh', 'zzMLC Guide', 'zzVOIAxilla And Skin']
    ['CTV INM', 'Heart', 'PTVn_LN_IMN_L', 'ptv imn dvh']

(left) z0k0kwmghm - study timestamp: 20211116_190607
    ['32Gy isodose', '36Gy isodose', '38Gy isodose', 'A', 'CTVn_IMN', 'CTVn_Level_1', 'CTVn_Level_2', 'CTVn_Level_3', 'CTVn_Level_4', 'CTVp_4000', 'CTVp_4000_DNU', 'Carotid Artery', 'Contra_Breast', 'Contra_Lung', 'Couch Top', 'FLASH', 'Heart', 'Heart_inField', 'HumeralHead_1cm', 'Humeralhead', 'Ipsi_Lung', 'LN_PTV_DVH', 'Oesophagus', 'PTV LN AX ONLY', 'PTV_Combined', 'PTV_WB_DVH', 'PTVn_LN_Ax_L1', 'PTVn_LN_Ax_L1_DVH', 'PTVn_LN_Ax_L2', 'PTVn_LN_Ax_L2_DVH', 'PTVn_LN_Ax_L3', 'PTVn

AssertionError: 

In [ ]:
# For selected patient, create registration with relapse scan as fixed image
idx = 0
p = patients[idx]

# Crop relapse scan to include structure-set ROIs plus margin.
ct_relapse = p.get_ct_relapse().clone()
ct_relapse.crop_to_roi(p.get_ss_relapse() + p.get_ss_recurrence(), buffer=100)

# Crop planning scan to size of cropped relapse scan.
ct_plan = p.get_ct_plan().clone()
ct_plan.crop_to_image(ct_relapse, alignment="spinal_canal")

# Define registration strategy.
reg = Registration(
    Path("results/relapse_fixed") / p.id,
    fixed=ct_relapse,
    moving=ct_plan,
    fixed_mask=None,
    initial_alignment = None,
    pfiles={
        #"translation": pfiles_dir / "MI_Translation.txt",
        "rigid": pfiles_dir / "MI_Rigid.txt",
        "affine": pfiles_dir / "MI_Affine.txt",
        #"bspline": pfiles_dir / "MI_BSpline30.txt"
    },
    overwrite=True,
    capture_output=True
)

In [ ]:
# Perform registration.
reg.register()

In [ ]:
# Show results at each step of registration.
for step in reg.steps:
    print(step)
    reg.view_result(step)

In [ ]:
# Push ROI contours from relapse frame to planning frame.
ss_relapse_transformed = reg.transform(p.get_ss_relapse(), transform_points=True)
ss_recurrence_transformed = reg.transform(p.get_ss_recurrence(), transform_points=True)
ss_recurrence_transformed["recurrence"].name = "Transformed_recurrence"

In [ ]:
# Show relapse scan with ROI contours outlined on scan,
# and ROI contours pushed from planning scan.
reg.moving_image.view(rois=ss_relapse_transformed + p.get_ss_plan())

In [ ]:
# Show recurrence mapped to planning scan and on relapse scan.
ct_plan.view(images=ct_relapse, rois=[ss_recurrence_transformed, p.get_ss_recurrence()])